In [7]:
from datetime import datetime

def get_raddr_csv(df_st,df_sst):
    df_st = df_st[['역명','도로명주소']]
    df_st['역명'] = df_st['역명'].str.replace('(', ' ',regex=False).str.split().str[0]
    df_sst = df_sst[['역명','도로명주소']]
    df_sst['역명'] = df_sst['역명'].str.replace('(', ' ',regex=False).str.split().str[0]
    df_st_r = pd.concat([df_sst, df_st], axis=0)
    df_st_r = df_st_r.reset_index(drop=True)
    now = datetime.now()
    f_date = now.strftime('%Y%m%d')
    df_st_r.to_csv(f'data/st_addr_{f_date}.csv',index=False,encoding='utf-8')
    return None

In [8]:
df_st = pd.read_csv('data/St_addr_20221122.csv',encoding='euc-kr')
df_sst = pd.read_csv('data/St_inseoul_addr.csv',encoding='euc-kr')
get_raddr_csv(df_st,df_sst)
# 평택지제 -> 지제
# 운천 삭제가능
# 쌍용 -> 쌍용동
# 명학 : 안안로 -> 만안로 /오타
# 응봉 : 124 -> 123
# 제물포 : 미추홀구로 변경/ 오타
# 세류역 : 주소뒤에 역사 제거

In [1]:
import numpy as np
import pandas as pd
import folium, requests
from folium.plugins import HeatMap
import requests,json,os
from urllib.parse import quote 

In [ ]:
# import folium
# from folium.plugins import HeatMap

# # 지도 생성
# m = folium.Map(location=[37.394946, 127.111104], zoom_start=17)

# # 데이터
# data = [(37.394946, 127.111104), (37.394957, 127.111115), (37.394968, 127.111126)]

# # HeatMap 추가
# HeatMap(data).add_to(m)

# # 지도 출력
# m
# #
# map = folium.Map(location=[37.541, 126.986],zoom_start=12)
# folium.Marker(
#     location=[37.541, 126.986],
    
#     tooltip='경기도청'
# ).add_to(map)
# folium.
# map

In [104]:
df = pd.read_csv('data/main/lines/1호선.csv')
df.head()

,사용월,호선명,지하철역,출근시간 하차인원,09-16시 승차인원,09-16시 하차인원,퇴근시간 승차인원,퇴근시간 하차인원,새벽 승차인원,새벽 하차인원,야간 승차인원,야간 하차인원,총 승차인원,총 하차인원
0,201501,1호선,가능,23287,96744,66805,31775,60948,8225,1240,17566,55647,190620,146979
1,201501,1호선,가산디지털단지,230457,107638,189559,209852,86319,8884,1544,114317,58461,274951,480021
2,201501,1호선,간석,15467,78718,55206,28721,55488,5554,1282,17595,56137,164392,128092
3,201501,1호선,개봉,52104,280814,191890,90855,217153,15614,3593,50829,184806,565027,432393
4,201501,1호선,관악,22169,100507,83845,40644,65829,6103,1307,19637,61074,210027,168395


In [5]:
df.지하철역[25]

'동대문'

In [3]:
# kakao_local_api 사용하기 위한 함수
def kakao_location(place):
    with open('../DataAnalysis/04.지도시각화/data/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [4]:
# 입력된 역의 좌표를 구하기 위해 역명을 입력받아 도로명 주소를 반환
def rtn_addr(target):
    str_addr = df_st[df_st.역명 == target].도로명주소.values[-1]
    return str_addr.strip()

In [105]:
df_st = pd.read_csv('data/st_addr_20230529.csv')

In [69]:
df_st.nunique()

역명       500
도로명주소    562
dtype: int64

In [25]:
target = df.지하철역[79].strip()
df_st[df_st.역명 == target].도로명주소.values[-1]

'서울특별시 성동구 고산자로 123  '

In [11]:
df.지하철역[50]

'세류'

In [12]:
a = rtn_addr(df.지하철역[50])
a

'경기도 수원시 권선구 정조로 393-1'

In [9]:
b = kakao_location(a)
b

(37.2446548766964, 127.01397825425)

In [106]:
df_st = pd.read_csv('data/st_addr_20230529.csv')
df_a = df[['호선명','지하철역']].copy()
df_a.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df_a.reset_index(drop=True,inplace=True)
df_a.tail()

,호선명,지하철역
0,1호선,가능
1,1호선,가산디지털단지
2,1호선,간석
3,1호선,개봉
4,1호선,관악


In [107]:
df_a.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)

In [108]:
df_a.reset_index(drop=True,inplace=True)
df_a.shape

(106, 2)

In [109]:
df_a.tail()

,호선명,지하철역
101,1호선,회룡
102,1호선,동두천중앙
103,1호선,쌍용
104,1호선,평택지제
105,1호선,탕정


In [110]:
df_a.nunique()

호선명       1
지하철역    106
dtype: int64

In [32]:
temp1 =[]
for i in df.index:
    try:
        target = df['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, df.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df = pd.concat([df, df_test], axis=1)
df.head()

50 세류
866 세류


In [111]:
# 여기서 부터
df_st = pd.read_csv('data/st_addr_20230529.csv')
df_a = df[['호선명','지하철역']].copy()
df_a.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df_a.reset_index(drop=True,inplace=True)
df_st.rename(columns={'역명':'지하철역'},inplace=True)
df_st.head()

,지하철역,도로명주소
0,서울,서울특별시 중구 세종대로 지하2(남대문로 5가)
1,시청,서울특별시 중구 세종대로 지하101(정동)
2,종각,서울특별시 종로구 종로 지하55(종로1가)
3,종로3가,서울특별시 종로구 종로 지하129(종로3가)
4,종로5가,서울특별시 종로구 종로 지하216(종로5가)


In [112]:
res = pd.merge(df_a,df_st,on='지하철역',how='left')
res

,호선명,지하철역,도로명주소
0,1호선,가능,경기도 의정부시 평화로 633
1,1호선,가산디지털단지,서울특별시 금천구 벚꽃로 309(가산동)
2,1호선,가산디지털단지,서울특별시 금천구 벚꽃로 309
3,1호선,간석,인천광역시 남동구 석정로 522-14
4,1호선,개봉,서울특별시 구로구 경인로40길 47
...,...,...,...
122,1호선,회룡,경기도 의정부시 평화로 363(회룡역)
123,1호선,동두천중앙,경기도 동두천시 동두천로 228
124,1호선,쌍용,충청남도 천안시 서북구 쌍용19로 20
125,1호선,평택지제,경기도 평택시 경기대로 777


In [113]:
res.to_csv('merge_test.csv',index=False)

In [114]:
def kakao_location(place):
    with open('../DataAnalysis/04.지도시각화/data/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [115]:
def rtn_addr(target):
    str_addr = df_st[df_st.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [117]:
temp1 =[]
for i in res.index:
    try:
        target = res['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, res.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
res = pd.concat([res, df_test], axis=1)


In [118]:
res.head()

,호선명,지하철역,도로명주소,lat,lng
0,1호선,가능,경기도 의정부시 평화로 633,37.748369,127.044221
1,1호선,가산디지털단지,서울특별시 금천구 벚꽃로 309(가산동),37.481596,126.882559
2,1호선,가산디지털단지,서울특별시 금천구 벚꽃로 309,37.481596,126.882559
3,1호선,간석,인천광역시 남동구 석정로 522-14,37.464688,126.693451
4,1호선,개봉,서울특별시 구로구 경인로40길 47,37.494522,126.859103


In [119]:
res.to_csv('latlng_test.csv',index=False)

In [126]:
res = pd.read_csv('latlng_test.csv')

In [127]:
res.drop(['도로명주소','호선명'],inplace=True, axis=1)
res.head()

,지하철역,lat,lng
0,가능,37.748369,127.044221
1,가산디지털단지,37.481596,126.882559
2,가산디지털단지,37.481596,126.882559
3,간석,37.464688,126.693451
4,개봉,37.494522,126.859103


In [128]:
result = pd.merge(df,res,on='지하철역',how='left')
result.head()

,사용월,호선명,지하철역,출근시간 하차인원,09-16시 승차인원,09-16시 하차인원,퇴근시간 승차인원,퇴근시간 하차인원,새벽 승차인원,새벽 하차인원,야간 승차인원,야간 하차인원,총 승차인원,총 하차인원,lat,lng
0,201501,1호선,가능,23287,96744,66805,31775,60948,8225,1240,17566,55647,190620,146979,37.748369,127.044221
1,201501,1호선,가산디지털단지,230457,107638,189559,209852,86319,8884,1544,114317,58461,274951,480021,37.481596,126.882559
2,201501,1호선,가산디지털단지,230457,107638,189559,209852,86319,8884,1544,114317,58461,274951,480021,37.481596,126.882559
3,201501,1호선,간석,15467,78718,55206,28721,55488,5554,1282,17595,56137,164392,128092,37.464688,126.693451
4,201501,1호선,개봉,52104,280814,191890,90855,217153,15614,3593,50829,184806,565027,432393,37.494522,126.859103


In [129]:
result.tail()

,사용월,호선명,지하철역,출근시간 하차인원,09-16시 승차인원,09-16시 하차인원,퇴근시간 승차인원,퇴근시간 하차인원,새벽 승차인원,새벽 하차인원,야간 승차인원,야간 하차인원,총 승차인원,총 하차인원,lat,lng
12393,202304,1호선,평택,56852,180388,179987,91683,117160,7697,3138,60755,64121,327486,304098,36.991897,127.083289
12394,202304,1호선,평택지제,35007,44321,32338,38983,27618,2261,10220,16797,21817,91106,99382,37.018785,127.070485
12395,202304,1호선,한남,31351,41848,44005,36938,21227,1913,829,18546,11256,75145,87441,37.529188,127.008250
12396,202304,1호선,화서,26927,88087,62869,32872,91560,8922,2608,13678,53588,216038,145992,37.284894,126.989200
12397,202304,1호선,회룡,27869,159633,118699,36597,127541,16541,1959,18364,81747,330387,230274,37.724422,127.047219
